<a href="https://colab.research.google.com/github/valentingorce/tp_centrale/blob/main/Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Information Retrieval
## Introduction to search engines

### DAY 2: Teacher version
### Implementing a search engine

The goal of this second session is to implement a first architecture of a search engine on the previously introduced dataset (stackexchange-datascience). If you missed the first session or if you did not saved the dataset, please reload the first session's notebook to download it. 

If you need some ifnormation about the dataset, it should be available here : https://archive.org/details/stackexchange

The notebook is divided into several steps:
-	Implement the indexation
-	Implement the search method
-	Define a ranking strategy and implement it
-	Suggest some improvements of the search engine



## Initialisation

In [1]:
# !pip install ttable

In [ ]:
import pandas as pd
import re
import os
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from tt import BooleanExpression
from itertools import product

In [ ]:
# # Only if you use Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
DATA_PATH = 'datascience.stackexchange.com/'


**Important :**

An Excel file for testing the evaluation part is available in the gitlab repo : evaluation_search_engine_post_queries_ranking_EI_CS.xlsx

If you work on Colab, we advice you to push it directly on your Google Drive directory.

# Implement the indexation
As you might already know, for a search engine to work properly an index of the documents must be created. Here we will keep it in python, and try to use only common libraries to keep it simple.

Once created, the index will be used to match the query with the documents. As a result, there are several ways to build an index, using statistical, boolean, semantic indexation...

First of, let's make a naive one that will consist in breaking down each document into a set of the words it contains.

In [ ]:
def extract_words(text):
  return text.split(' ')



In [ ]:

# test
s = "The cat is sat on the mat. The dog is laid on the mat."
words = extract_words(s)
assert sorted(words) == ['The', 'The', 'cat', 'dog', 'is', 'is', 'laid', 'mat.', 'mat.', 'on', 'on', 'sat', 'the', 'the']

As you may notice, there are several problems with the previous implementation. First, "The" and "the" aren't considered the same, the "." is kept at the the end of "mat." as any other punctuation character... 

Re-implement this function with some basic preprocessing to avoid these issues.

In [ ]:
# problems : First, "The" and "the" aren't considered the same, the "." is kept at the the end of "mat." as any other punctuation character... 
def extract_words(text:str)->list:
  text = text.lower()
  text = re.sub(r'[^\w\s]','',text)
  return text.split(' ')

In [ ]:
# test
print(sorted(extract_words(s)))
assert sorted(extract_words(s))==['cat', 'dog', 'is', 'is', 'laid', 'mat', 'mat', 'on', 'on', 'sat', 'the', 'the', 'the', 'the']

Now you sould be able to create your index table. For now we will just make a dataframe with two columns: [raw_text, words].

In [ ]:
import pandas as pd
def index_docs(docs:list[str])->pd.DataFrame:
  df = pd.DataFrame(docs,columns=['raw_text'])
  df['words'] = df['raw_text'].apply(extract_words)
  return df

In [ ]:
# test

L = [s, "Hello World!", "Goodbye", "How are you?"]

index_docs(L)

Now, let's try it on the dataset:

In [ ]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

For our first version of the indexation mechanism, we will simply use the "body" of the posts. To have a better search engine, the title and other metadata aswell could be used aswell. Finally, not all the XML files have a "body" feature, so for the search engine to retrieve information from any of the files you will need to implement another way to index.

But first, let's start with "body". There is more to preprocess than before, indeed, there are html tags such as "<p>" for instance. They are not useful for us, because users won't use them in their queries. So we first need to remove them.

In [ ]:
def remove_tags(text:str)->str:
  return re.sub(r'<[^>]+>', '', text)


In [ ]:
# test
remove_tags('<p>Hello World!\nI am making a search engine.<p>')

In [ ]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)
clean_posts

In [ ]:
clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)
clean_posts

## Zipf Law

A way of analyzing a corpus is to draw the zipf law

In [ ]:
# Draw Zipf Law on the Posts Corpus
import matplotlib.pyplot as plt
# get word count for each word
word_count = clean_posts['words'].explode().value_counts()
word_count

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(word_count.values)
plt.xlabel('Word Rank')
plt.ylabel('Word Count')
# log-log scale
plt.yscale('log')
plt.xscale('log')
plt.title('Zipf Law')
plt.show()

## Inverted Index

Now, we want to go further on the indexing and build an inverted index. Inverted index is a dictionary where the keys are the words of the vocabulary and the values are the documents containing these words. Reducing the size of the vocabulary is a relevant first step when building an inverted index. Here, we will focus on the creation of the index, we leave you the optimisation steps :)

In [ ]:
def create_index(posts:pd.DataFrame):
  index = {}
  for _, row in posts.iterrows():
    for word in row['words']:
      if word in index:
        index[word].add(row['Id'])
      else:
        index[word] = {row['Id']}
  return index

In [ ]:
# inverted_index = create_index(clean_posts.iloc[0:5000])
# # save index to pickle
# import pickle
# with open('inverted_index.pickle', 'wb') as handle:
#     pickle.dump(inverted_index, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# load index from pickle
import pickle
inverted_index = {}
with open('inverted_index.pickle', 'rb') as handle:
    inverted_index = pickle.load(handle)

#### Well Done, you've indexed the dataset! 
Don't hesitate to save your indexes in txt or pickle file

---
# Implement the search method

A naive method would be to count the number of words in common between the query and each posts. Then to rank the posts you could directly select the post who maximize the number of common words. Let's implement this approach :

In [ ]:
# Implement the word_in_index function 
# Inputs : a word (str) & a list of words
# Output : pandas series of 1 if the word is in the list, else 0

def word_in_index(word, word_list_index):
  return pd.Series([1 if word in words else 0 for words in word_list_index])

# test
print(word_in_index('cat', [['cat', 'dog'], ['cat', 'mouse'], ['dog', 'mouse']]))


In [ ]:
# Implement the function which run through a pandas series and count the number of word in common
# Use extract_words method, apply method with word_in_index function
# Inputs : the query (str) & pandas series of strings
# Output : Pandas series counting the number of common words between the query and each string in word_serie

def count_common_words(query, word_serie):
  query_words = extract_words(query)
  return word_serie.apply(lambda x: sum(word_in_index(word, [x]) for word in query_words))

# test
print(count_common_words('cat dog', pd.Series([['cat', 'dog'], ['cat', 'mouse'], ['dog', 'mouse']])))



In [ ]:

def rank_top_query(query, df, top=5):
  # get the number of common words between the query and each document
  common_words = count_common_words(query, df['words'])
  # sort the documents by number of common words
  sorted_common_words = common_words.sort_values(by=0, ascending=False)
  # return the top documents
  return df.iloc[sorted_common_words.index[0:top]]


In [ ]:
results = rank_top_query(query="testing the query in python", df=clean_posts, top=5) # prends 1min30 pour tourner


In [ ]:
# print(results)
for _, row in results.iterrows():
    print(row['Clean Body'])
    print('-------------------')

In [ ]:
# pros:
# - easy to implement
# - fast to compute (1min30 par requete, bon...)
# cons:
# - gives the same weight to all words, even common words like "the" or "is"
# - doesn't take into account the order of the words in the query
# - doesn't take into account the order of the words in the documents
# - doesn't take into account the number of times a word appears in a document
# - doesn't take into account the number of times a word appears in the corpus


Testez plusieurs requêtes et critiquez les résultats obtenus.

Quels sont les pros and cons de cette méthodes. Vous l'indiquerez sur le rapport avec vos réflexions pour l'améliorer.

Next, you have to implement the first improvements you find in the search method to get most relevant results 

In [ ]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stop_words =  stopwords.words('english')

In [ ]:

def remove_stop_words(l_txt: list) -> list:
    return [word for word in l_txt if word not in stop_words]

# test
print(remove_stop_words(['the', 'cat', 'is', 'on', 'the', 'mat'])) # ['cat', 'mat']

## Boolean Search

Thanks to the ttable library, implement a boolean search method

In [ ]:
from tt import BooleanExpression

def boolean_search(query, df=clean_posts):
  # get the words in the query
  expression =  BooleanExpression(query)
  # get the posts whose clean body that satisfy the expression
  symbols = expression.symbols
  # for each post and for each symbol, check if the symbol is in the post, then evaluate the expression
  bools = df['Clean Body'].apply(lambda x: [symbol in x for symbol in symbols]).apply(lambda x: expression.evaluate(**dict(zip(symbols,x))))
  # return all documents that satisfy the expression
  return df.iloc[bools[bools].index]

  


  # return the top documents

results = boolean_search('java AND NOT python')

for _, row in results[0:5].iterrows():
    print(row['Clean Body'])
    print('-------------------')

## Probabilistic search

Implement the MIB or BM25 method of searching

In [ ]:
print(clean_posts)

In [ ]:
def proba_search(query, df=clean_posts, top=5):
  # each document get a score
  # OKAPI model (BM25)
  # print('query : ', query)
  # print('top : ', top)
  query_words = extract_words(query)
  k1 = 1.2
  b = 0.75
  k3 = 1000
  # average length of a document
  m = df['Clean Body'].apply(lambda x: len(x)).mean()
  N = len(df)
  RSV_score = {}
  # for each post in df :
  for _, row in df.iterrows():
    # sum over all words in the query and in the post
    # length of the post
    Ld = len(row['Clean Body'])
    # term frequency in the query
    def tf(word):
      return sum([1 for w in query_words if w == word])
    def d_f(word):
      if word not in inverted_index:
        #print(word)
        return 0
      else:
        return len(inverted_index[word])
    RSV_score[row['Id']] = sum([(k1+1)*tf(word)/(k1*((1-b)+b*Ld/m)+tf(word))*(k3+1)*tf(word)/(k3+tf(word))*np.log((N-d_f(word)+0.5)/(d_f(word)+0.5)) for word in query_words if word in row['words']])
  # return the top Ids of the posts from RSV
  sorted_keys = sorted(RSV_score, key=RSV_score.get, reverse=True) # the Id column in the best order
  # the values of sorted_keys are values of df["Id"]
  # return the top documents in the same order as sorted_keys
  new_df = df.copy()
  new_df['RSV_score'] = new_df['Id'].apply(lambda x: RSV_score[x])
  new_df = new_df.sort_values(by='RSV_score', ascending=False)
  return new_df[new_df["Id"].isin(sorted_keys[0:top])]


In [ ]:
# test
print("begin")
results = proba_search('measure performance for multi classification model',top=len(clean_posts))
# reset index


In [ ]:
print(results)

# for _, row in results.iterrows():
#     print(row['Clean Body'])
#     print('-------------------')

Compare the naive method with your improvements and the boolean and probabilistic search. (report)



---



---




# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.



We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the xlsx file : . The final score will be the mean quadratic error of the queries.


Explication for the xlsx file :

We propose you a Excel file with some posts and a mesure of relevancy for the queries

- First column is the post Id,
- Columns starting by query are the queries you have to test.
- The values in this columns are the rank of relevancy of the post in regard with the query.
- The missing values indicates you should not take into account the post


You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [2]:
# Read Relevancy CSV
import pandas as pd
df_relevancy = pd.read_excel("evaluation_search_engine_post_queries_ranking_EI_CS.xlsx")
df_relevancy.head()

,PostId,Title,First Sentence,Query 1 : mesure performance for multiclassification model,Query 2 : draw neural network,Query 3 : neural network layers,Query 4 : how sklearn working,Query 5 : treat categorical data
0,6107,What are deconvolutional layers?,I recently read Fully Convolutional Networks f...,NaN,NaN,1.0,NaN,NaN
1,15989,Micro Average vs Macro average Performance in ...,I am trying out a multiclass classification se...,1.0,NaN,NaN,NaN,NaN
2,13490,How to set class weights for imbalanced classe...,I know that there is a possibility in Keras wi...,3.0,NaN,NaN,NaN,NaN
3,12321,What's the difference between fit and fit_tran...,I do not understand the difference between the...,NaN,NaN,NaN,1.0,3.0
4,22,K-Means clustering for mixed numeric and categ...,My data set contains a number of numeric attri...,NaN,NaN,NaN,5.0,2.0


In [ ]:
from sklearn.metrics import ndcg_score

    """
    Calculates the NDCG (Normalized Discounted Cumulative Gain) score for each query in the given relevance dataframe
    using the specified search method.

    Parameters:
    -----------
    df_relevancy : pandas.DataFrame
        A dataframe containing the relevance scores for each post and query.
        The first column should contain the post IDs, and the remaining columns from the 4th should be named 'query X',
        where X is the query number starting from 1.
        The values in the columns should be the relevance scores for each post with respect to the corresponding query.
    method : function
        The search method to use for retrieving the top documents for each query.
        The function should take a query string as input and return a pandas.DataFrame containing the top documents.
    top : int, optional
        The number of top documents to retrieve for each query.
        The default value is the total number of posts in the dataset.

    Returns:
    --------
    dict
        A dictionary containing the NDCG score for each query.
        The keys are the query numbers starting from 1, and the values are the corresponding NDCG scores.
    """


def get_ndcg_scores(df_relevancy,method=proba_search,top=len(clean_posts)):
    # for each PostId in the relevancy dataframe, get the rank of the post according to the method
    querys = {}
    for i in range(3, 8):
        querys[i] = df_relevancy.columns[i][10:]
    method_results = {}
    for i in range(3, 8):
        method_results[i] = method(query=querys[i],top=top)
        #print(method_results[i].head())
    # get the score of each post according to the method
    rel_preds = {}
    for i in range(3, 8):
        # the rankings of the posts in PostId
        rel_preds[i] = [list(method_results[i]["Id"]).index(x) if x in method_results[i]["Id"] else top for x in df_relevancy['PostId']]
        #print(rel_preds[i])
    rel_trues = {}
    for i in range(3, 8):
        # the rankings of the posts
        rel_trues[i] = df_relevancy[df_relevancy.columns[i]].tolist()
        #print(rel_trues[i])
        rel_trues[i] = [top if np.isnan(x) else x for x in rel_trues[i]]

    # calculate the ndcg score for each query
    ndcg_scores = {}
    for i in range(3, 8):
        ndcg_scores[i] = ndcg_score([rel_trues[i]], [rel_preds[i]])
    return ndcg_scores
    

print(get_ndcg_scores(df_relevancy,method=proba_search))
   
# rel_pred = 


# table of ndcg for each query
# ndcg_table = pd.DataFrame(columns=['Query', 'NDCG'])
# for i in range(4, 8):
  # ndcg_table = ndcg_table.append({'Query': f'Query {i-3}', 'NDCG': calculate_ndgc(df_relevancy.columns[i])}, ignore_index=True)

# print(ndcg_table)